# Preprocessing datasets

File dedicated to preprocessing datasets to preprare them for use in the final model. 

In [1]:
# importing packages
import pandas as pd
import os
import problem

## US GDP data

In [2]:
us_gdp = pd.read_csv(os.path.join('raw_add_data', 'gdp_data.csv'), na_values='(D)')
us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN


The first step is to associate the column GeoName with the list of airports from the original problem. 

In [3]:
cities = us_gdp['GeoName'].unique()
print(f'Total cities: {len(cities)}')
print(f'The cities are: \n{cities}')


Total cities: 18
The cities are: 
['Atlanta-Sandy Springs-Alpharetta, GA (Metropolitan Statistical Area)'
 'Austin-Round Rock-Georgetown, TX (Metropolitan Statistical Area)'
 'Boston-Cambridge-Newton, MA-NH (Metropolitan Statistical Area)'
 'Charlotte-Concord-Gastonia, NC-SC (Metropolitan Statistical Area)'
 'Chicago-Naperville-Elgin, IL-IN-WI (Metropolitan Statistical Area)'
 'Denver-Aurora-Lakewood, CO (Metropolitan Statistical Area) *'
 'Detroit-Warren-Dearborn, MI (Metropolitan Statistical Area)'
 'Houston-The Woodlands-Sugar Land, TX (Metropolitan Statistical Area)'
 'Las Vegas-Henderson-Paradise, NV (Metropolitan Statistical Area)'
 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)'
 'Miami-Fort Lauderdale-Pompano Beach, FL (Metropolitan Statistical Area)'
 'Minneapolis-St. Paul-Bloomington, MN-WI (Metropolitan Statistical Area)'
 'New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)'
 'Orlando-Kissimmee-Sanford, FL (Metropolitan Statistical Are

In [4]:
#retrieve name of the first city from GeoName column
us_gdp['city'] = us_gdp['GeoName'].apply(lambda city: city.split('-')[0])
cities = us_gdp['city'].unique()
print('The values in the column city are:\n', cities) 

The values in the column city are:
 ['Atlanta' 'Austin' 'Boston' 'Charlotte' 'Chicago' 'Denver' 'Detroit'
 'Houston' 'Las Vegas' 'Los Angeles' 'Miami' 'Minneapolis' 'New York'
 'Orlando' 'Philadelphia' 'Phoenix' 'San Francisco' 'Seattle']


We can check the correspondance between the cities and the airports from the file `city_ariport.csv`

In [5]:
city_airport = pd.read_csv(os.path.join('raw_add_data', 'city_airport.csv'), header=None)
city_airport.columns = ['airport', 'city']
print(city_airport)

   airport           city
0      ATL        Atlanta
1      BOS         Boston
2      CLT      Charlotte
3      ORD        Chicago
4      DEN         Denver
5      DTW        Detroit
6      DFW         Austin
7      IAH        Houston
8      LAS      Las Vegas
9      LAX    Los Angeles
10     MIA          Miami
11     MSP    Minneapolis
12     JFK       New York
13     LGA       New York
14     EWR       New York
15     MCO        Orlando
16     PHL   Philadelphia
17     PHX        Phoenix
18     SFO  San Francisco
19     SEA        Seattle


In [6]:
merged_us_gdp = pd.merge(us_gdp, city_airport, on='city')
merged_us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013,city,airport
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0,Atlanta,ATL
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0,Atlanta,ATL
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0,Atlanta,ATL
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0,Atlanta,ATL
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN,Atlanta,ATL


The columns `city`, `GeoFips` and `GeoName` are no longer useful. The column `Description` is encode in `LineCode` and can also be deleted.

In [7]:
del (merged_us_gdp['GeoFips'], merged_us_gdp['GeoName'], merged_us_gdp['Description'],
     merged_us_gdp['city'])


In [8]:
merged_us_gdp.head()

,LineCode,2011,2012,2013,airport
0,1.0,277405509.0,287570424.0,299918186.0,ATL
1,2.0,250779823.0,260962774.0,273113553.0,ATL
2,3.0,NaN,NaN,602096.0,ATL
3,6.0,267771.0,228417.0,231368.0,ATL
4,10.0,3704264.0,3582854.0,NaN,ATL


In [9]:
#melting data frame to obtain years as a factor
melted_us_gdp = merged_us_gdp.melt(id_vars=['LineCode', 'airport'], 
                                   value_vars=['2011', '2012', '2013'],
                                   var_name='year', value_name='gdp')
melted_us_gdp.head()

,LineCode,airport,year,gdp
0,1.0,ATL,2011,277405509.0
1,2.0,ATL,2011,250779823.0
2,3.0,ATL,2011,NaN
3,6.0,ATL,2011,267771.0
4,10.0,ATL,2011,3704264.0


In [10]:
#converting dates to datetime
melted_us_gdp['year'] = pd.to_datetime(melted_us_gdp['year'], format='%Y')
melted_us_gdp.set_index('year', inplace=True)
melted_us_gdp.head()

,LineCode,airport,gdp
year,,,
2011-01-01,1.0,ATL,277405509.0
2011-01-01,2.0,ATL,250779823.0
2011-01-01,3.0,ATL,NaN
2011-01-01,6.0,ATL,267771.0
2011-01-01,10.0,ATL,3704264.0


In [11]:
melted_us_gdp.to_csv(os.path.join('processed_add_data', 'gdp_data_processed.csv'))

## US holiday data

In [12]:
us_hol = pd.read_csv(os.path.join('raw_add_data', 'usholidays.csv'), usecols=['Date', 'Holiday'])
us_hol['Date'] = pd.to_datetime(us_hol['Date']) # convert dates to datetime
us_hol.set_index('Date', inplace=True)
us_hol.head()

,Holiday
Date,
2010-12-31,New Year's Day
2011-01-17,"Birthday of Martin Luther King, Jr."
2011-02-21,Washington's Birthday
2011-05-30,Memorial Day
2011-07-04,Independence Day


To select the relevant dates, we rely on the train and test datasets

In [13]:
x_train, _ = problem.get_train_data()
x_test, _ = problem.get_test_data()

max_date = max(x_test['DateOfDeparture'].max(), x_train['DateOfDeparture'].max())
min_date = min(x_test['DateOfDeparture'].min(), x_train['DateOfDeparture'].min())

print(f'The max date is: {max_date}')
print(f'The min date is: {min_date}')

The max date is: 2013-03-05
The min date is: 2011-09-01


In [14]:
us_hol = us_hol[min_date : max_date]
us_hol.to_csv(os.path.join('processed_add_data', 'usholidays_processed.csv'))

## US demographics - working pop

In [15]:
#https://stats.oecd.org/Index.aspx?DataSetCode=CITIES
#gives working population (15-65 yo) by metropolitan areas

In [16]:
#loading and renaming dataset
working_pop = pd.read_csv(os.path.join('raw_add_data', 'working_age_pop.csv'), usecols=['Metropolitan areas', 'Year', 'Value'])
working_pop = working_pop.rename(columns={'Metropolitan areas':'city', 'Year':'year', 'Value':'work_pop'})
working_pop.head()

,city,year,work_pop
0,New York (Greater),2011,13415543
1,New York (Greater),2012,13425475
2,New York (Greater),2013,13435986
3,Los Angeles (Greater),2011,11781575
4,Los Angeles (Greater),2012,11854951


In [17]:
#clean metropolitan area names 
working_pop['city'] = working_pop['city'].str.replace("Greater", "")
working_pop['city'] = working_pop['city'].str.replace("\\(\\)", "")
working_pop['city'] = working_pop['city'].str.rstrip()

In [18]:
#merge working_pop to city_airport to get airport belonging to metro_area
working_pop = pd.merge(working_pop, city_airport, how='left',  on='city')
working_pop.head()

,city,year,work_pop,airport
0,New York,2011,13415543,JFK
1,New York,2011,13415543,LGA
2,New York,2011,13415543,EWR
3,New York,2012,13425475,JFK
4,New York,2012,13425475,LGA


In [19]:
working_pop.to_csv(os.path.join('processed_add_data', 'working_pop.csv'))

## US demographics - gender

In [20]:
age_sex = pd.read_csv(os.path.join('raw_add_data', 'demo_age_sex.csv'), usecols=['NAME', 'SEX', 'AGE', 'POPEST2011_CIV', 'POPEST2012_CIV', 'POPEST2013_CIV' ])
age_sex = age_sex.rename(columns={'NAME':'state', 'SEX':'sex', 'AGE':'age', 'POPEST2011_CIV':'2011', 'POPEST2012_CIV':'2012', 'POPEST2013_CIV':'2013'})
age_sex.head()

,state,sex,age,2011,2012,2013
0,United States,0,0,3963092,3926570,3931258
1,United States,0,1,3966225,3977549,3942698
2,United States,0,2,3970654,3978925,3991740
3,United States,0,3,4101644,3981531,3991017
4,United States,0,4,4121488,4111490,3992502


In [21]:
#keep only relevant states by merging with list of airports needed
#create table of Airports and State
#source: https://www.transtats.bts.gov/tables.asp?Table_ID=288&SYS_Table_Name=T_MASTER_CORD
airport_state = pd.read_csv(os.path.join('raw_add_data', 'airport_city_state.csv'), usecols=['AIRPORT', 'AIRPORT_STATE_NAME'])
airport_state = airport_state.rename(columns={'AIRPORT':'airport','AIRPORT_STATE_NAME':'state'})
airport_state = airport_state.drop_duplicates(subset=['airport'])
airport_state.head()

,airport,state
0,01A,Alaska
1,03A,Alaska
2,04A,Alaska
3,05A,Alaska
4,06A,Alaska


In [22]:
#merge to city_airport to get a list of airport-city-state
airport_city_state = pd.merge(city_airport, airport_state, how='left', on='airport')
airport_city_state.head()

,airport,city,state
0,ATL,Atlanta,Georgia
1,BOS,Boston,Massachusetts
2,CLT,Charlotte,North Carolina
3,ORD,Chicago,Illinois
4,DEN,Denver,Colorado


In [23]:
#merge age_sex to airport_city_state, to only keep necessary airports
airport_sex_state = pd.merge(airport_city_state, age_sex, how='left', on='state')

#only keep age==999 --> total population
airport_sex_state = airport_sex_state[airport_sex_state['age']==999]

#only keep sex is 1(male) or 2(female) 
airport_sex_state = airport_sex_state[airport_sex_state['sex']!=0]

In [24]:
#melting data frame to obtain years as a factor
melted_sex = airport_sex_state.melt(value_vars=['2011', '2012', '2013'],
                                    id_vars=['airport', 'city', 'state', 'sex'],
                                    var_name='year', value_name='population')
melted_sex = melted_sex.sort_values(by=['airport', 'sex'])

#table with seperated pop_female and pop_male
sex_male = melted_sex[melted_sex['sex']==1].rename(columns={'population':'pop_male'}).drop(['sex'], axis=1)
sex_female = melted_sex[melted_sex['sex']==2].rename(columns={'population':'pop_female'}).drop(['sex'], axis=1)
pop_by_sex = pd.merge(sex_female, sex_male, how='left', on=['airport', 'year', 'city', 'state'])
pop_by_sex.head()

,airport,city,state,year,pop_female,pop_male
0,ATL,Atlanta,Georgia,2011,5005893,4728362
1,ATL,Atlanta,Georgia,2012,5057673,4778824
2,ATL,Atlanta,Georgia,2013,5098814,4810147
3,BOS,Boston,Massachusetts,2011,3411504,3194591
4,BOS,Boston,Massachusetts,2012,3433819,3223186


In [25]:
pop_by_sex.to_csv(os.path.join('processed_add_data', 'pop_by_sex.csv'))

## Fuel Prices

US Gulf coast Kerosene type fuel price. We use it to proxy the price variation

In [26]:
#import data
fuel = pd.read_csv(os.path.join('raw_add_data', 'fuel_prices.csv'))

In [27]:
fuel.head()

,Date,JET_FUEL_TYPE (Dollars per Gallon)
0,"Apr 02, 1990",0.550
1,"Apr 03, 1990",0.555
2,"Apr 04, 1990",0.560
3,"Apr 05, 1990",0.540
4,"Apr 06, 1990",0.536


In [28]:
fuel['Date'] = pd.to_datetime(fuel['Date'])

In [29]:
fuel.set_index('Date', inplace=True)

Similarly to the US Holidays dataset, we are only interested in a portion of the dates

In [30]:
fuel = fuel[min_date : max_date] #dates retrieved from problem.py 

In [31]:
fuel.to_csv(os.path.join('processed_add_data', 'fuel_prices.csv'))

## US flights data

The data comes from the Bureau of Transportation Statistics' T-100 form. It aggregates the domestic flight infomartion monthly by Air Company in the US.

In [32]:
flight_11 = pd.read_csv(os.path.join('raw_add_data', 'flight_data_2011.csv'))
flight_12 = pd.read_csv(os.path.join('raw_add_data', 'flight_data_2012.csv'))
flight_13 = pd.read_csv(os.path.join('raw_add_data', 'flight_data_2013.csv'))

In [33]:
flight = pd.concat([flight_11, flight_12, flight_13])

### Data Exploration

In [34]:
print(flight.columns)

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'PAYLOAD', 'SEATS',
       'PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'RAMP_TO_RAMP', 'AIR_TIME',
       'CARRIER_NAME', 'ORIGIN', 'DEST', 'YEAR', 'MONTH', 'Unnamed: 15'],
      dtype='object')


In [35]:
print(flight['Unnamed: 15'].unique())
del flight['Unnamed: 15']

[nan]


In [36]:
del (flight['PAYLOAD'], flight['FREIGHT'], flight['MAIL'], flight['RAMP_TO_RAMP'],
     flight['AIR_TIME'], flight['CARRIER_NAME'])

In [37]:
print(flight.columns)

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS',
       'DISTANCE', 'ORIGIN', 'DEST', 'YEAR', 'MONTH'],
      dtype='object')


In [38]:
flight.head()

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,ORIGIN,DEST,YEAR,MONTH
0,0.0,2.0,0.0,0.0,0.0,LRD,LRD,2011,9
1,0.0,1.0,0.0,0.0,0.0,LRD,LRD,2011,9
2,0.0,1.0,0.0,0.0,855.0,LRD,MCI,2011,9
3,0.0,1.0,0.0,0.0,1562.0,LRD,MDT,2011,9
4,0.0,1.0,0.0,0.0,762.0,LRD,MEM,2011,9


In [39]:
test1 = flight['ORIGIN']=='LAX'
test2 = flight['DEST']=='JFK'
test3 = flight['MONTH']==3
test4 = flight['YEAR']==2011
flight.loc[test1 & test2 & test3 & test4, :]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,ORIGIN,DEST,YEAR,MONTH
37842,0.0,26.0,6096.0,3978.0,2475.0,LAX,JFK,2011,3
37863,0.0,1.0,0.0,0.0,2475.0,LAX,JFK,2011,3
128734,1.0,1.0,225.0,133.0,2475.0,LAX,JFK,2011,3
155024,2.0,2.0,376.0,355.0,2475.0,LAX,JFK,2011,3
327119,89.0,90.0,13500.0,11681.0,2475.0,LAX,JFK,2011,3
348607,181.0,181.0,26921.0,23399.0,2475.0,LAX,JFK,2011,3
348691,182.0,182.0,20020.0,17479.0,2475.0,LAX,JFK,2011,3
350167,209.0,209.0,36361.0,29610.0,2475.0,LAX,JFK,2011,3
351998,302.0,300.0,50400.0,43643.0,2475.0,LAX,JFK,2011,3


Since our original data was divided by air carrier, we have more than one entry per month. We need, therefore, to group the data and sum the columns DEPARTURES_SCHEDULED, DEPARTURES_PERFORMED, SEATS, and PASSENGERS

In [40]:
flight = flight.groupby(['MONTH', 'YEAR', 'ORIGIN', 'DEST', 'DISTANCE'], as_index=False).sum()

In [41]:
#check that we have only one entry per month
# redefine tests as indexes were changed
test1 = flight['ORIGIN']=='LAX'
test2 = flight['DEST']=='JFK'
test3 = flight['MONTH']==3
test4 = flight['YEAR']==2011
flight.loc[test1 & test2 & test3 & test4, :]

,MONTH,YEAR,ORIGIN,DEST,DISTANCE,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS
66042,3,2011,LAX,JFK,2475.0,966.0,992.0,153899.0,130278.0


### Data Transformation
#### Route Importance 
We want to calculate the total number of passengers, flights, and seats that originated from each of the airports.

In [42]:
origin_totals = flight[['MONTH', 'YEAR', 'ORIGIN', 'PASSENGERS']].groupby(
    ['MONTH', 'YEAR', 'ORIGIN'], as_index=False
                                                                    ).sum()
origin_totals.rename(columns={'PASSENGERS': 'TOT_PAS_MON_ORIGIN'}, inplace=True)
origin_totals.head()

,MONTH,YEAR,ORIGIN,TOT_PAS_MON_ORIGIN
0,1,2011,1G4,664.0
1,1,2011,A07,0.0
2,1,2011,A23,7.0
3,1,2011,A27,8.0
4,1,2011,AA8,3.0


In [43]:
flight = pd.merge(flight, origin_totals, on=['MONTH', 'YEAR', 'ORIGIN'], how='outer')
flight.head()

,MONTH,YEAR,ORIGIN,DEST,DISTANCE,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,TOT_PAS_MON_ORIGIN
0,1,2011,1G4,VGT,79.0,0.0,49.0,931.0,664.0,664.0
1,1,2011,A07,OTZ,68.0,0.0,1.0,0.0,0.0,0.0
2,1,2011,A23,HOM,19.0,0.0,9.0,54.0,7.0,7.0
3,1,2011,A27,FAI,91.0,0.0,4.0,36.0,8.0,8.0
4,1,2011,AA8,SCC,34.0,0.0,1.0,19.0,3.0,3.0



We also want to include the importance of the route for the origin airport. That is: the quotient of PASSENGERS and TOT_PAS_MON_ORIGIN.

In [44]:
flight['IMPORT_ROUTE'] = flight['PASSENGERS']/flight['TOT_PAS_MON_ORIGIN']

#### Importance of the date

In [45]:
month_totals = flight[['MONTH', 'YEAR', 'PASSENGERS']].groupby(
    ['MONTH', 'YEAR'], as_index=False
                                                                    ).sum()
month_totals.rename(columns={'PASSENGERS': 'MONTH_PAS'}, inplace=True) 
month_totals.head(10)

,MONTH,YEAR,MONTH_PAS
0,1,2011,47642716.0
1,1,2012,48415831.0
2,1,2013,48866403.0
3,2,2011,44842780.0
4,2,2012,47518997.0
5,2,2013,46653322.0
6,3,2011,57119109.0
7,3,2012,57540740.0
8,3,2013,57715217.0
9,4,2011,54301108.0


In [46]:
year_totals = month_totals[['YEAR', 'MONTH_PAS']].groupby(
    ['YEAR'], as_index=False).sum()
year_totals.rename(columns={'MONTH_PAS': 'YEAR_PAS'}, inplace=True) 
year_totals.head()

,YEAR,YEAR_PAS
0,2011,656320870.0
1,2012,657645308.0
2,2013,660057640.0


In [47]:
importance_df = pd.merge(month_totals, year_totals, how='left', on=['YEAR'])
importance_df['IMPORT_MONTH'] = importance_df['MONTH_PAS']/importance_df['YEAR_PAS']
importance_df.head()

,MONTH,YEAR,MONTH_PAS,YEAR_PAS,IMPORT_MONTH
0,1,2011,47642716.0,656320870.0,0.072591
1,1,2012,48415831.0,657645308.0,0.073620
2,1,2013,48866403.0,660057640.0,0.074034
3,2,2011,44842780.0,656320870.0,0.068324
4,2,2012,47518997.0,657645308.0,0.072256


In [48]:
# add it to flight dataset
flight = flight.merge(importance_df, on=['MONTH', 'YEAR'], how='left')

#### Load and cancelling rate

In [49]:
#calculate load factor
flight['LOAD'] = flight['PASSENGERS']/flight['SEATS']
#calcualte cancelling rate
flight['CANCEL_RATE'] = flight['DEPARTURES_PERFORMED']/flight['DEPARTURES_SCHEDULED']

In [50]:
#correcting for unscheduled flights performed
flight.loc[flight['CANCEL_RATE'] > 1, 'CANCEL_RATE'] = 1

### Export Data

In [51]:

flight.to_csv(os.path.join('processed_add_data', 'flight_data.csv'))